In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
offset = 0
limit = 100000
period = 'max'# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [3]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()

print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11128


In [4]:
# Validate and convert symbols to strings
symbols = [str(s) if not isinstance(s, str) else s for s in symbols]
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11128


In [5]:
! pip install yfinance > /dev/null 2>&1
! mkdir hist

In [6]:
import yfinance as yf
import os, contextlib

In [7]:
import os
import time
import contextlib
import yfinance as yf

# Initialize timer
start_time = time.time()

# Parameters (set these according to your needs)
limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)

# Create a directory for storing CSV files if it doesn't exist
os.makedirs('hist', exist_ok=True)

# Track progress
downloaded_count = 0

# Force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            downloaded_count += 1
            data.to_csv(f'hist/{s}.csv')

            # Print progress every 10 files
            if downloaded_count % 10 == 0:
                elapsed_time = time.time() - start_time
                print(f"{downloaded_count} files downloaded so far. Elapsed time: {elapsed_time:.2f} seconds.")

# Total elapsed time
total_time = time.time() - start_time
print(f"Total number of valid symbols downloaded = {sum(is_valid)}")
print(f"Total time taken: {total_time:.2f} seconds.")

# Check how many files are in the folder
file_count = len([f for f in os.listdir('hist') if os.path.isfile(os.path.join('hist', f))])
print(f"Number of files in the 'hist' folder: {file_count}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total number of valid symbols downloaded = 10212
Total time taken: 2789.82 seconds.
Number of files in the 'hist' folder: 10212


In [8]:
print(f"Length of symbols: {len(symbols)}")
print(f"Length of is_valid: {len(is_valid)}")
print(f"Number of rows in data_clean: {len(data_clean)}")

Length of symbols: 11128
Length of is_valid: 11128
Number of rows in data_clean: 11128


In [9]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

In [10]:
!mkdir stocks
!mkdir etfs

In [11]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [12]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [13]:
import os

# Specify the folder path
folder_path = 'stocks'  # Replace with the path to your folder

# Count the number of files
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of files in the folder '{folder_path}': {file_count}")

Number of files in the folder 'stocks': 6283


In [22]:
import os
import pandas as pd

# Define the folder containing the stock files
input_folder = "./stocks"  # Replace with your folder path
output_folder = "./Stocks"  # Define the output folder

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each CSV file in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        # Full path to the input file
        file_path = os.path.join(input_folder, filename)

        # Extract the ticker name from the file name (e.g., NVDA from NVDA.csv)
        ticker_name = os.path.splitext(filename)[0]
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Skip if the file doesn't have enough rows
        if len(df) <= 2:
            print(f"Skipping {filename}: Not enough rows.")
            continue
        
        # Remove the first two rows
        df = df.iloc[2:].reset_index(drop=True)
        
        # Rename the "Price" column to "Date"
        if "Price" in df.columns:
            df = df.rename(columns={"Price": "Date"})
        
        # Add a new "Symbol" column populated with the ticker name
        df['Symbol'] = ticker_name
        
        # Save the modified file to the output folder
        output_file_path = os.path.join(output_folder, filename)
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {filename}")

print("All files processed!")


Processed and saved: PERF.csv
Processed and saved: PFG.csv
Processed and saved: ELAN.csv
Processed and saved: ZG.csv
Processed and saved: SLE.csv
Processed and saved: STEM.csv
Processed and saved: RANGU.csv
Processed and saved: SPSC.csv
Processed and saved: BV.csv
Processed and saved: LILA.csv
Processed and saved: AIMAU.csv
Processed and saved: GHC.csv
Processed and saved: BNGO.csv
Processed and saved: DEC.csv
Processed and saved: EVR.csv
Processed and saved: CX.csv
Processed and saved: CVLG.csv
Processed and saved: XXII.csv
Processed and saved: COHN.csv
Processed and saved: FLUX.csv
Processed and saved: SVRA.csv
Processed and saved: BLACU.csv
Processed and saved: CRWS.csv
Processed and saved: ATLO.csv
Processed and saved: BBWI.csv
Processed and saved: IZEA.csv
Processed and saved: BKKT.csv
Processed and saved: PWUP.csv
Processed and saved: LITB.csv
Processed and saved: BDL.csv
Processed and saved: TXRH.csv
Processed and saved: BTOC.csv
Processed and saved: FPH.csv
Processed and saved:

In [23]:
df = pd.read_csv("./Stocks/NVDA.csv")
df

,Date,Adj Close,Close,High,Low,Open,Volume,Symbol
0,1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000,NVDA
1,1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000,NVDA
2,1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000,NVDA
3,1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000,NVDA
4,1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000,NVDA
...,...,...,...,...,...,...,...,...
6537,2025-01-16,133.570007,133.570007,138.750000,133.490005,138.639999,209235600,NVDA
6538,2025-01-17,137.710007,137.710007,138.500000,135.460007,136.690002,201188800,NVDA
6539,2025-01-21,140.830002,140.830002,141.830002,137.089996,139.160004,197749000,NVDA
6540,2025-01-22,147.070007,147.070007,147.789993,143.669998,144.660004,237651400,NVDA


In [24]:
# Define the folder containing the stock files
input_folder = "./etfs"  # Replace with your folder path
output_folder = "./ETFs"  # Define the output folder

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each CSV file in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        # Full path to the input file
        file_path = os.path.join(input_folder, filename)

        # Extract the ticker name from the file name 
        ticker_name = os.path.splitext(filename)[0]
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Skip if the file doesn't have enough rows
        if len(df) <= 2:
            print(f"Skipping {filename}: Not enough rows.")
            continue
        
        # Remove the first two rows
        df = df.iloc[2:].reset_index(drop=True)
        
        # Rename the "Price" column to "Date"
        if "Price" in df.columns:
            df = df.rename(columns={"Price": "Date"})
        
        # Add a new "Symbol" column populated with the ticker name
        df['Symbol'] = ticker_name
        
        # Save the modified file to the output folder
        output_file_path = os.path.join(output_folder, filename)
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {filename}")

print("All files processed!")


Processed and saved: AUSF.csv
Processed and saved: PCY.csv
Processed and saved: PLTM.csv
Processed and saved: BAUG.csv
Processed and saved: AMID.csv
Processed and saved: PSWD.csv
Processed and saved: IVRA.csv
Processed and saved: HYD.csv
Processed and saved: THD.csv
Processed and saved: FDEV.csv
Processed and saved: BRKD.csv
Processed and saved: IXP.csv
Processed and saved: UTES.csv
Processed and saved: SFY.csv
Processed and saved: HYRM.csv
Processed and saved: LTL.csv
Processed and saved: MSMR.csv
Processed and saved: PSK.csv
Processed and saved: BILZ.csv
Processed and saved: SPMB.csv
Processed and saved: EPS.csv
Processed and saved: XMLV.csv
Processed and saved: AGZ.csv
Processed and saved: CEFA.csv
Processed and saved: AGRH.csv
Processed and saved: DFAW.csv
Processed and saved: HERD.csv
Processed and saved: IBIK.csv
Processed and saved: IGEB.csv
Processed and saved: IPPP.csv
Processed and saved: BITS.csv
Processed and saved: XLG.csv
Processed and saved: NUSB.csv
Processed and saved:

In [26]:
df = pd.read_csv("./ETFs/TQQQ.csv")
df

,Date,Adj Close,Close,High,Low,Open,Volume,Symbol
0,2010-02-11,0.416216,0.432552,0.434896,0.405573,0.406875,3456000,TQQQ
1,2010-02-12,0.417920,0.434323,0.438073,0.418333,0.420781,8601600,TQQQ
2,2010-02-16,0.434108,0.451146,0.452188,0.437552,0.444531,9619200,TQQQ
3,2010-02-17,0.441475,0.458802,0.458906,0.450313,0.457188,19180800,TQQQ
4,2010-02-18,0.449844,0.467500,0.470260,0.455573,0.458333,38860800,TQQQ
...,...,...,...,...,...,...,...,...
3756,2025-01-16,79.230003,79.230003,82.050003,79.199997,82.010002,46527700,TQQQ
3757,2025-01-17,83.139999,83.139999,83.940002,81.839996,83.690002,43676100,TQQQ
3758,2025-01-21,84.519997,84.519997,85.150002,82.330002,84.449997,41725900,TQQQ
3759,2025-01-22,87.800003,87.800003,88.910004,86.709999,86.839996,43132700,TQQQ


In [29]:
! rmdir hist

In [37]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [38]:
zip_dir()

/kaggle/working/directory.zip